# ETL Process

Short description

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set(color_codes=True)

# MYSQL to Python

In [2]:
import pymysql as sql
from credentials import credentials

## Stablishing a connection

In [3]:
user = credentials.get('USERNAME')
passwd = credentials.get('PASSWORD')

In [4]:
# creating connection
con = sql.connect(
                host = '127.0.0.1', user = user, passwd = passwd) #optional - db="schema_name"  

In [5]:
cursor = con.cursor()

## Executing SQL statements

In order to run MySQL comands within Python, we need to use the **execute()** function. <br> <br>

A Database has been created from MySQL Workbench, so now let's create a table from Python. <br>
After specifying which DB to use and writing the Table's attributes, we must **commit** for the commands to take action.

In [6]:
cursor.execute("USE drinksDB;")
cursor.execute("DROP TABLE IF EXISTS Drinks;")
cursor.execute("""
CREATE TABLE IF NOT EXISTS Drinks(
    Country VARCHAR(50) NOT NULL,
    Beer_servings INT NOT NULL,
    Spirit_servings INT NOT NULL,
    Wine_servings INT NOT NULL,
    Total_litres_of_pure_alcohol FLOAT NOT NULL,
    Continent VARCHAR(2) );
""")
               
               


0

In [7]:
con.commit()

## INSERT

Before we insert the values, we need to clean the data. Let's start by using Pandas to read the csv file.

In [8]:
raw_data = pd.read_csv('drinks.csv')
df = raw_data.copy()
df.head()

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
0,Afghanistan,0,0,0,0.0,AS
1,Albania,89,132,54,4.9,EU
2,Algeria,25,0,14,0.7,AF
3,Andorra,245,138,312,12.4,EU
4,Angola,217,57,45,5.9,AF


Now, observe that there are 23 null values in the *continent* column. We need to assign valid values to them.

In [9]:
df.isnull().sum()

country                          0
beer_servings                    0
spirit_servings                  0
wine_servings                    0
total_litres_of_pure_alcohol     0
continent                       23
dtype: int64

In [10]:
df['continent'].unique()

array(['AS', 'EU', 'AF', nan, 'SA', 'OC'], dtype=object)

Seems like **SA** represents *South America*, and we see no **NA**. So, let's visualize the null values to be sure.

In [11]:
df.query('continent != continent')

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
5,Antigua & Barbuda,102,128,45,4.9,NaN
11,Bahamas,122,176,51,6.3,NaN
14,Barbados,143,173,36,6.3,NaN
17,Belize,263,114,8,6.8,NaN
32,Canada,240,122,100,8.2,NaN
41,Costa Rica,149,87,11,4.4,NaN
43,Cuba,93,137,5,4.2,NaN
50,Dominica,52,286,26,6.6,NaN
51,Dominican Republic,193,147,9,6.2,NaN
54,El Salvador,52,69,2,2.2,NaN


There seems to be only American countries in the list, so let's give it the right values.

In [12]:
for i in [32, 109, 184]:
    df['continent'][i] = 'NA'
for i in list( set(df.query('continent != continent').index) - set([32, 109, 184])  ):
    df['continent'][i] = 'SA'

C:\Users\manol\AppData\Local\Temp\ipykernel_1716\1375426894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['continent'][i] = 'NA'
C:\Users\manol\AppData\Local\Temp\ipykernel_1716\1375426894.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['continent'][i] = 'SA'


In [13]:
df.query('continent == "NA"')

,country,beer_servings,spirit_servings,wine_servings,total_litres_of_pure_alcohol,continent
32,Canada,240,122,100,8.2,NA
109,Mexico,238,68,5,5.5,NA
184,USA,249,158,84,8.7,NA


Now, we need to create every single entry to INSERT it into the DB. We do the following: <br>
1) iterate throw df's rows.
2) Create a list containing each value in the row. (**arr**)
3) Join such list creating a separation with a coma. (**','.join(arr)**)
4) Create a list containing each of the entries. (**valueList**)
5) Join the list, separating each value with a coma. (**','.join(valueList)**)
6) Add a semicolon to indicate the last value to include and make the resulting string be called **values**.

In [14]:
valueList = []

colNames = list(df.columns)
for index,row in df.iterrows():
    arr=['"' + str(row[col]) + '"' for col in colNames]
    valueList.append( '(' + ','.join(arr) + ')' )

values = ', '.join(valueList)+';'
values


'("Afghanistan","0","0","0","0.0","AS"), ("Albania","89","132","54","4.9","EU"), ("Algeria","25","0","14","0.7","AF"), ("Andorra","245","138","312","12.4","EU"), ("Angola","217","57","45","5.9","AF"), ("Antigua & Barbuda","102","128","45","4.9","SA"), ("Argentina","193","25","221","8.3","SA"), ("Armenia","21","179","11","3.8","EU"), ("Australia","261","72","212","10.4","OC"), ("Austria","279","75","191","9.7","EU"), ("Azerbaijan","21","46","5","1.3","EU"), ("Bahamas","122","176","51","6.3","SA"), ("Bahrain","42","63","7","2.0","AS"), ("Bangladesh","0","0","0","0.0","AS"), ("Barbados","143","173","36","6.3","SA"), ("Belarus","142","373","42","14.4","EU"), ("Belgium","295","84","212","10.5","EU"), ("Belize","263","114","8","6.8","SA"), ("Benin","34","4","13","1.1","AF"), ("Bhutan","23","0","0","0.4","AS"), ("Bolivia","167","41","8","3.8","SA"), ("Bosnia-Herzegovina","76","173","8","4.6","EU"), ("Botswana","173","35","35","5.4","AF"), ("Brazil","245","145","16","7.2","SA"), ("Brunei","31"

Finally, create the INSERT query with a separator (placeholder), execute it substituting %s by **values** and commit.

In [15]:
query = """
INSERT INTO drinks values %s
""" %(values)

In [16]:
cursor.execute("""
INSERT INTO drinks values %s
""" % values)

193

In [17]:
con.commit()